In [1]:
import numpy as np
from projection import projection
import matplotlib.pyplot as plt
import cv2

# Extrinsic parameters
#   The world coordinate system is based on the mechanical CAD tool from Andre Adams
headset_params = {
    'World': {
        'Origin': [0.0, 0.0, 0.0],                  # meters
        'Direction': [0.0, 0.0, 0.0]                # degrees
    },
    'Camera': {
        'Origin': [-68.586e-3, 17.225e-3, 0.0],     # Top left of camera
        'Direction': [-6, 0.0, 0.0]
        #'Direction': [0.0, 0.0, 0.0]
    },
    'Left display': {
        'Origin': [-38.865e-3, 0.0, 0.0],           # Top left of display
        'Direction': [-10.032, -7.594, 0.0]
        #'Direction': [0.0, 0.0, 0.0]
    },
    'Right display': {
        'Origin': [38.865e-3, 0.0, 0.0],            # Top left of display
        'Direction': [-10.032, 7.594, 0.0]
        #'Direction': [0.0, 0.0, 0.0]
    }
}

# Camera intrinsic parameters
cam_params = {
    'Sensor': {
        'Name': 'IMX681',
        'Resolution': {
            'Hor': 3024,
            'Ver': 4032
        },
        'Dimension': {
            'Hor': 3.024e-3,
            'Ver': 4.032e-3
        },
        'Principal Point': {
            'Hor': 1512,
            'Ver': 2016
        }
    },
    'Optics': {
        'FoV': {
            'Hor': 70,
            'Ver': 92
        },
        'Focal Length': 100
    }
}

# Left and right displays intrinsic parameters
#   We assume the pupil is aligned with to the display's center
disp_params = {
    'Panel': {
        'Name': 'Mozaic v2',
        'Resolution': {
            'Hor': 48,
            'Ver': 40
        },
        'Dimension': {
            'Hor': 48e-3,
            'Ver': 40e-3
        },
        'Principal Point': {
            'Hor': 24,
            'Ver': 20
        }
    },
    'Eye': {
        'Relief': 15    # This is in pixel! 1 pixel = 1mm
    }
}

## Render Simulation

In [2]:
def add_line(renderer, start_point, end_point, color=(0, 0, 0), thickness=1.0):
    """
    Draws a line in the VTK renderer.

    Parameters:
    renderer : vtkRenderer
        The VTK renderer to which the line will be added.
    start_point : tuple of float
        The (x, y, z) coordinates of the start point of the line.
    end_point : tuple of float
        The (x, y, z) coordinates of the end point of the line.
    color : tuple of float
        The RGB color of the line (default is black).
    thickness : float
        The thickness of the line (default is 1.0).
    """
    # Create a line source
    line_source = vtk.vtkLineSource()
    line_source.SetPoint1(start_point)
    line_source.SetPoint2(end_point)

    # Create a mapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(line_source.GetOutputPort())

    # Create an actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(color)  # Set line color
    actor.GetProperty().SetLineWidth(thickness)  # Set line thickness

    # Add the actor to the renderer
    renderer.AddActor(actor)

def add_circle(renderer, center, radius, direction, color=(0, 0, 0), thickness=1.0, opacity=1.0):
    """
    Draws a circle in the VTK renderer.

    Parameters:
    renderer : vtkRenderer
        The VTK renderer to which the circle will be added.
    center : tuple of float
        The (x, y, z) coordinates of the center of the circle.
    radius : float
        The radius of the circle.
    direction : tuple of float
        The (x, y, z) direction vector normal to the plane of the circle.
    color : tuple of float
        The RGB color of the circle (default is black).
    thickness : float
        The thickness of the circle (default is 1.0).
    opacity : float
        The opacity of the circle (default is 1.0, fully opaque).
    """
    # Create a plane to define the circle's orientation
    plane = vtk.vtkPlaneSource()
    plane.SetOrigin(center)
    plane.SetPoint1(center[0] + radius, center[1], center[2])
    plane.SetPoint2(center[0], center[1] + radius, center[2])

    # Compute the normal of the plane
    normal = np.array(direction)
    normal = normal / np.linalg.norm(normal)  # Normalize the direction vector

    # Create a transform to rotate the plane
    transform = vtk.vtkTransform()
    transform.Translate(center)
    transform.RotateWXYZ(90, normal)

    # Create a circular pattern
    circle_source = vtk.vtkRegularPolygonSource()
    circle_source.SetNumberOfSides(100)
    circle_source.SetRadius(radius)
    circle_source.SetCenter(0, 0, 0)

    # Transform the circle
    transform_filter = vtk.vtkTransformPolyDataFilter()
    transform_filter.SetTransform(transform)
    transform_filter.SetInputConnection(circle_source.GetOutputPort())
    transform_filter.Update()

    # Create a mapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(transform_filter.GetOutputPort())

    # Create an actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(color)  # Set circle color
    actor.GetProperty().SetLineWidth(thickness)  # Set circle thickness
    actor.GetProperty().SetOpacity(opacity)  # Set circle opacity

    # Add the actor to the renderer
    renderer.AddActor(actor)

def add_triangle(renderer, point1, point2, point3, color=(0, 0, 0), opacity=1.0):
    """
    Draws a triangle in the VTK renderer.

    Parameters:
    renderer : vtkRenderer
        The VTK renderer to which the triangle will be added.
    point1 : tuple of float
        The (x, y, z) coordinates of the first vertex of the triangle.
    point2 : tuple of float
        The (x, y, z) coordinates of the second vertex of the triangle.
    point3 : tuple of float
        The (x, y, z) coordinates of the third vertex of the triangle.
    color : tuple of float
        The RGB color of the triangle (default is black).
    opacity : float
        The opacity of the triangle (default is 1.0, fully opaque).
    """
    # Create a points object and add the three points to it
    points = vtk.vtkPoints()
    points.InsertNextPoint(point1)
    points.InsertNextPoint(point2)
    points.InsertNextPoint(point3)

    # Create a cell array to store the triangle in
    triangles = vtk.vtkCellArray()

    # Create a triangle
    triangle = vtk.vtkTriangle()
    triangle.GetPointIds().SetId(0, 0)
    triangle.GetPointIds().SetId(1, 1)
    triangle.GetPointIds().SetId(2, 2)

    # Add the triangle to the list of triangles
    triangles.InsertNextCell(triangle)

    # Create a polydata object
    trianglePolyData = vtk.vtkPolyData()

    # Add the points to the dataset
    trianglePolyData.SetPoints(points)

    # Add the triangles to the dataset
    trianglePolyData.SetPolys(triangles)

    # Create a mapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(trianglePolyData)

    # Create an actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(color)  # Set triangle color
    actor.GetProperty().SetOpacity(opacity)  # Set triangle opacity

    # Add the actor to the renderer
    renderer.AddActor(actor)

def add_rectangle(renderer, point1, point2, point3, point4, color=(0, 0, 0), opacity=1.0):
    """
    Draws a rectangle in the VTK renderer, by drawing two triangles.

    Parameters:
    renderer : vtkRenderer
        The VTK renderer to which the triangle will be added.
    point1 : tuple of float
        The (x, y, z) coordinates of the first vertex.
    point2 : tuple of float
        The (x, y, z) coordinates of the second vertex.
    point3 : tuple of float
        The (x, y, z) coordinates of the third vertex.
    point4 : tuple of float
        The (x, y, z) coordinates of the fourth vertex.
    color : tuple of float
        The RGB color of the rectangle (default is black).
    opacity : float
        The opacity of the rectangle (default is 1.0, fully opaque).
    """
    add_triangle(renderer, point1, point2, point4, color, opacity)
    add_triangle(renderer, point2, point3, point4, color, opacity)
    
def add_sphere(renderer, center, radius, color=(0, 0, 0), opacity=1.0, phi_resolution=100, theta_resolution=100):
    """
    Add a sphere to the renderer at the specified center with the specified radius and resolution.
    
    Parameters:
    - renderer: The vtkRenderer instance.
    - center: A tuple of (x, y, z) coordinates for the center of the sphere.
    - radius: The radius of the sphere.
    - phi_resolution: Number of subdivisions along the latitude (default is 100).
    - theta_resolution: Number of subdivisions along the longitude (default is 100).
    """
    # Create a sphere source
    sphere_source = vtk.vtkSphereSource()
    sphere_source.SetCenter(center)
    sphere_source.SetRadius(radius)
    sphere_source.SetPhiResolution(phi_resolution)
    sphere_source.SetThetaResolution(theta_resolution)
    
    # Create a mapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(sphere_source.GetOutputPort())
    
    # Create an actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(color)  # Set sphere color
    actor.GetProperty().SetOpacity(opacity)  # Set sphere opacity
    
    # Add the actor to the renderer
    renderer.AddActor(actor)

def add_cylinder(renderer, center, radius, direction, height, color=(0, 0, 0), opacity=1.0, resolution=100):
    """
    Draws a cylinder in the VTK renderer.

    Parameters:
    renderer : vtkRenderer
        The VTK renderer to which the cylinder will be added.
    center : tuple of float
        The (x, y, z) coordinates of the center of the base of the cylinder.
    radius : float
        The radius of the cylinder.
    direction : tuple of float
        The (x, y, z) direction vector along the height of the cylinder.
    height : float
        The height of the cylinder.
    color : tuple of float
        The RGB color of the cylinder (default is black).
    opacity : float
        The opacity of the cylinder (default is 1.0, fully opaque).
    resolution : int
        The number of sides of the cylinder (default is 100).
    """
    # Create a cylinder source
    cylinder_source = vtk.vtkCylinderSource()
    cylinder_source.SetRadius(radius)
    cylinder_source.SetHeight(height)
    cylinder_source.SetResolution(resolution)

    # Compute the normalized direction vector
    direction = np.array(direction)
    direction = direction / np.linalg.norm(direction)

    # Compute the axis of rotation
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(z_axis, direction)
    rotation_angle = np.degrees(np.arccos(np.dot(z_axis, direction)))

    # Create a transform to orient the cylinder
    transform = vtk.vtkTransform()
    transform.Translate(center)
    transform.RotateWXYZ(rotation_angle, rotation_axis)

    # Transform the cylinder
    transform_filter = vtk.vtkTransformPolyDataFilter()
    transform_filter.SetTransform(transform)
    transform_filter.SetInputConnection(cylinder_source.GetOutputPort())
    transform_filter.Update()

    # Create a mapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(transform_filter.GetOutputPort())

    # Create an actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(color)  # Set cylinder color
    actor.GetProperty().SetOpacity(opacity)  # Set cylinder opacity

    # Add the actor to the renderer
    renderer.AddActor(actor)
    
def add_cuboid(renderer, center, size):
    """
    Add a cuboid to the renderer at the specified center with the specified size.
    
    Parameters:
    - renderer: The vtkRenderer instance.
    - center: A tuple of (x, y, z) coordinates for the center of the cube.
    - size: The size of the cuboid (x, y then z axes).
    """
    # Create a cube source
    cube_source = vtk.vtkCubeSource()
    #half_size = size / 2.0
    cube_source.SetXLength(size[0])
    cube_source.SetYLength(size[1])
    cube_source.SetZLength(size[2])
    cube_source.SetCenter(center)
    
    # Create a mapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(cube_source.GetOutputPort())
    
    # Create an actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    
    # Add the actor to the renderer
    renderer.AddActor(actor)

def add_frustum(renderer, frustum, color=(0, 0, 0)):
    [pt0, pt1, pt2, pt3, pt4] = frustum
    add_sphere(renderer, center=pt0, radius=1e-3, color=color, opacity=1)
    add_rectangle(renderer, point1=pt1, point2=pt2, point3=pt3, point4=pt4, color=color, opacity=1.0)
    add_line(renderer, start_point=pt0, end_point=pt1, color=color)
    add_line(renderer, start_point=pt0, end_point=pt2, color=color)
    add_line(renderer, start_point=pt0, end_point=pt3, color=color)
    add_line(renderer, start_point=pt0, end_point=pt4, color=color)
    add_line(renderer, start_point=pt1, end_point=pt2, color=color)
    add_line(renderer, start_point=pt2, end_point=pt3, color=color)
    add_line(renderer, start_point=pt3, end_point=pt4, color=color)
    add_line(renderer, start_point=pt4, end_point=pt1, color=color)

def plot_single_projection_error(depths, pt2d_ldisp, pt2d_rdisp):
    scale = 25
    xmax, ymax = 48, 40
    img = np.ones((ymax * scale, xmax * scale, 3), dtype=np.uint8)*255
    for x in range(48):
        cv2.line(img, (scale * x, 0), (scale * x, ymax * scale), (224, 224, 224))

    for y in range(40):
        cv2.line(img, (0, scale * y), (xmax * scale, scale * y), (224, 224, 224))

    cv2.putText(img, 'Camera-display projection error', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(img, 'Object located on camera at coordinate [150, 2016]', (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(img, 'Object range 250mm to 5m', (50, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(img, 'Left display', (250, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 1)
    cv2.putText(img, 'Right display', (750, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 1)
    for i, depth in enumerate(depths.reshape(10, 1)):
        xl = int(pt2d_ldisp[0, i] * scale)
        yl = int(pt2d_ldisp[1, i] * scale)
        cv2.circle(img, (xl, yl), 5, (255, 0, 0), -1)
        xr = int(pt2d_rdisp[0, i] * scale)
        yr = int(pt2d_rdisp[1, i] * scale)
        cv2.circle(img, (xr, yr), 5, (0, 0, 255), -1)

    cv2.imshow('Himax display', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Single Point Operation

In [3]:
if False:
    supernova = projection(headset_params, cam_params, disp_params)

    pt2d = np.array([1512, 2016])
    pt2d = np.array([200, 248])
    #print(f'pt2d_0: {pt2d_0.shape}')
    #print(f'pt2d_0: {pt2d_0}\r\n')
    # Convert the pixel point to homogeneous coordinates
    pixel_point_homogeneous = np.array([pt2d[0], pt2d[1], 1.0])
    #print(f'pixel_point_homogeneous: {pixel_point_homogeneous.shape}')
    print(f'pixel_point_homogeneous: {pixel_point_homogeneous}\r\n')

    # Compute the inverse of the intrinsic matrix
    intrinsic_matrix_inv = np.linalg.inv(supernova.K_cam)
    #print(f'intrinsic_matrix_inv: {intrinsic_matrix_inv.shape}')
    print(f'intrinsic_matrix_inv: {intrinsic_matrix_inv}\r\n')

    # Transform the pixel point to normalized image coordinates
    normalized_coords = intrinsic_matrix_inv @ pixel_point_homogeneous
    #print(f'normalized_coords: {normalized_coords.shape}')
    print(f'normalized_coords: {normalized_coords}\r\n')

    # The ray direction in camera coordinates is the normalized image coordinates
    ray_direction = normalized_coords / np.linalg.norm(normalized_coords)
    #print(f'ray_direction: {ray_direction.shape}')
    print(f'ray_direction: {ray_direction}\r\n')
    ray_direction = ray_direction.reshape(3, 1)

    dmin, dmax, dstep = 250e-3, 5000e-3, 10
    depths = np.geomspace(dmin, dmax, dstep)
    pt3d = ray_direction  * depths[7]
    #print(f'pt3d: {pt3d.shape}')
    print(f'pt3d: {pt3d}\r\n')
    pt3d_w = np.linalg.inv(supernova.R_cam)@(pt3d - supernova.t_w_cam)
    #print(f'pt3d_w: {pt3d_w.shape}')
    print(f'pt3d_w: {pt3d_w}\r\n')

    pt3d_ldisp = supernova.R_ldisp@pt3d_w + supernova.t_w_ldisp
    #print(f'pt3d_ldisp: {pt3d_ldisp.shape}')
    print(f'pt3d_ldisp: {pt3d_ldisp}\r\n')
    pt2d_ldisp = supernova.K_disp @ pt3d_ldisp
    print(f'pt2d_ldisp: {pt2d_ldisp}\r\n')
    pt2d_ldisp = pt2d_ldisp[:2] / pt2d_ldisp[2]
    #print(f'pt2d_ldisp: {pt2d_ldisp.shape}')
    print(f'pt2d_ldisp: {pt2d_ldisp}\r\n')

# Multiple Points Operation

In [ ]:
supernova = projection(headset_params, cam_params, disp_params)

x_size, y_size = 189, 252
xarr = np.tile(np.arange(1, x_size + 1), (y_size, 1)) * 16 - 8   # (252, 189)
yarr = np.tile(np.arange(1, y_size + 1), (x_size, 1)).T * 16 - 8 # (252, 189)
merged_xy = np.stack((yarr, xarr), axis=2)    # (252, 189, 2)
pt2d = merged_xy.reshape(-1, 2)  #(252 * 189, 2)

# Convert the pixel point to homogeneous coordinates
ones_column = np.ones((pt2d.shape[0], 1))   # (252 * 189, 1)
pixel_point_homogeneous = np.hstack((pt2d, ones_column))    # (252 * 189, 3)

# Compute the inverse of the intrinsic matrix
intrinsic_matrix_inv = np.linalg.inv(supernova.K_cam)   # (3, 3)

# Transform the pixel point to normalized image coordinates
normalized_coords = (intrinsic_matrix_inv @ pixel_point_homogeneous.T).T    # (252 * 189, 3)

# Compute ray vector intersecting camera optical center and 2d points on image plane
euclidian_distance = np.linalg.norm(normalized_coords, axis=1).reshape(normalized_coords.shape[0],1)    # (252 * 189, 1)
ray_direction = normalized_coords / euclidian_distance   # (252 * 189, 3)

# Generate 3d points along rays
dmin, dmax, dstep = 250e-3, 5000e-3, 10
depths = np.geomspace(dmin, dmax, dstep)    # (dstep, )
depths = depths[:, np.newaxis, np.newaxis]  # (dstep, 1, 1)
pt3d = ray_direction  * depths  # (dstep, 252 * 189, 3)
pt3d = pt3d.reshape(-1, 3)  # (dstep * 252 * 189, 3)

# Tranform 3d points to world coordinate
pt3d_w = (np.linalg.inv(supernova.R_cam)@(pt3d - supernova.t_w_cam.T).T).T  # (dstep * 252 * 189, 3)

# Tranform 3d points to display 3d and 2d coordinate
#   Left
pt3d_ldisp = (supernova.R_ldisp@pt3d_w.T + supernova.t_w_ldisp).T   # (dstep * 252 * 189, 3)
pt2d_ldisp = (supernova.K_disp @ pt3d_ldisp.T).T    # (dstep * 252 * 189, 3)
third_elements = pt2d_ldisp[:, 2].reshape(pt2d_ldisp.shape[0], 1)   # (dstep * 252 * 189, 1)
pt2d_ldisp = pt2d_ldisp / third_elements    # (dstep * 252 * 189, 3)
pt2d_ldisp = np.delete(pt2d_ldisp, 2, axis=1)   # (dstep * 252 * 189, 2)
pt2d_ldisp = pt2d_ldisp.reshape(dstep, y_size, x_size, 2)   # (dstep, 252, 189, 2)
#   Right
pt3d_rdisp = (supernova.R_rdisp@pt3d_w.T + supernova.t_w_rdisp).T   # (dstep * 252 * 189, 3)
pt2d_rdisp = (supernova.K_disp @ pt3d_rdisp.T).T    # (dstep * 252 * 189, 3)
third_elements = pt2d_rdisp[:, 2].reshape(pt2d_rdisp.shape[0], 1)   # (dstep * 252 * 189, 1)
pt2d_rdisp = pt2d_rdisp / third_elements    # (dstep * 252 * 189, 3)
pt2d_rdisp = np.delete(pt2d_rdisp, 2, axis=1)   # (dstep * 252 * 189, 2)
pt2d_rdisp = pt2d_rdisp.reshape(dstep, y_size, x_size, 2)   # (dstep, 252, 189, 2)

# Compute projection error
x = depths[:, 0, 0]   # (10, )

pt2d_ldisp[pt2d_ldisp < 0] = 0      # Make sure all coordinates are positive
pt2d_ldisp[..., 0] = np.clip(pt2d_ldisp[..., 0], None, 39)  # Make sure all y coordinates are smaller than the resolution of the display
pt2d_rdisp[pt2d_rdisp < 0] = 0
pt2d_ldisp[..., 1] = np.clip(pt2d_ldisp[..., 1], None, 47)  # Make sure all x coordinates are smaller than the resolution of the display

# Assume the projection from camera to display of the points at depth = 250mm and 5000mm have the largest Euclidian distance from each other
pt0 = pt2d_rdisp[0, :, :, :]    # (252, 189, 2)
pt1 = pt2d_rdisp[-1, :, :, :]   # (252, 189, 2)
a = np.linalg.norm(pt1 - pt1, axis=2)   # (252, 189)
#a = (a - a.min()) / (a.max() - a.min())
print(a)

print(f'a: {a.shape}')
cv2.imshow('projection error', a)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Test
d, x, y = 2, 94, 126
print(f'The coordinate [{xarr[0, x]}, {yarr[y, 0]}] on the camera plane at depth {depths[d, 0]} meters projects to the left display plane at coordinate {pt2d_ldisp[d, x, y, :]}')
print(f'The coordinate [{xarr[0, x]}, {yarr[y, 0]}] on the camera plane at depth {depths[d, 0]} meters projects to the right display plane at coordinate {pt2d_rdisp[d, x, y, :]}')

# All in One Marix

In [ ]:
supernova = projection(headset_params, cam_params, disp_params)

#[pt2d_cam, depths, pt2d_ldisp, pt2d_rdisp] = supernova.compute_projection_error(np.array([supernova.K_cam[0, 2], supernova.K_cam[1, 2]]), [250e-3, 5000e-3, 10])
[pt2d_cam, depths, pt2d_ldisp, pt2d_rdisp] = supernova.compute_projection_error(np.array([150, 2016]), [250e-3, 5000e-3, 10])

#plot_single_projection_error(depths, pt2d_ldisp, pt2d_rdisp)


x = np.tile(np.arange(1, 252 + 1), (189, 1)) * 16 - 8   # (189, 252)
y = np.tile(np.arange(1, 189 + 1), (252, 1)).T * 16 - 8 # (189, 252)
merged_xy = np.stack((x, y), axis=2)    # (189, 252, 2)
pt2d = merged_xy.reshape(-1, 2)  #(189 * 252, 2)

dmin, dmax, dstep = 250e-3, 5000e-3, 10
depths = np.geomspace(dmin, dmax, dstep).reshape(dstep, 1)    # (dstep, 1)
depths = depths[:, np.newaxis]  # (dstep, 1, 1)

ones_column = np.ones((pt2d.shape[0], 1))   # (189 * 252, 1)
pixel_point_homogeneous = np.hstack((pt2d, ones_column))    # (189 * 252, 3)
        
# Compute the inverse of the intrinsic matrix
intrinsic_matrix_inv = np.linalg.inv(supernova.K_cam)   # (3, 3)

# Transform the pixel point to normalized image coordinates
normalized_coords = (intrinsic_matrix_inv @ pixel_point_homogeneous.T).T    # (189 * 252, 3)

# The ray direction in camera coordinates is the normalized image coordinates
ray_direction = normalized_coords / np.linalg.norm(normalized_coords)   # (189 * 252, 3)
pt3d = ray_direction  * depths  # (dstep, 189 * 252, 3)

pt3d = pt3d.reshape(-1, 3)  # (dstep * 189 * 252, 3)
pt3d_w_ldisp = (supernova.R_ldisp@pt3d.T + supernova.t_w_ldisp).T   # (dstep * 189 * 252, 3)
pt3d_w_rdisp = (supernova.R_rdisp@pt3d.T + supernova.t_w_rdisp).T   # (dstep * 189 * 252, 3)

pt2d_ldisp = (supernova.K_disp @ pt3d_w_ldisp.T).T  # (dstep * 189 * 252, 3)
pt2d_ldisp = pt2d_ldisp / pt2d_ldisp[:, 2][:, np.newaxis]  # (dstep * 189 * 252, 3)
pt2d_rdisp = (supernova.K_disp @ pt3d_w_rdisp.T).T  # (dstep * 189 * 252, 3)
pt2d_rdisp = pt2d_rdisp / pt2d_rdisp[:, 2][:, np.newaxis]  # (dstep * 189 * 252, 3)

print(f'pt2d_ldisp: {pt2d_ldisp[0, :]}')

In [ ]:
import vtk
from projection import projection
import matplotlib.pyplot as plt

supernova = projection(headset_params, cam_params, disp_params)

# Create a renderer, render window, and interactor
renderer = vtk.vtkRenderer()
renderer.SetBackground(255, 255, 255)  # Background color
renderer.SetUseDepthPeeling(1)
renderer.SetMaximumNumberOfPeels(100)
renderer.SetOcclusionRatio(0.1)

renderWindow = vtk.vtkRenderWindow()
renderWindow.SetAlphaBitPlanes(1)  # Use alpha bit-planes
renderWindow.SetMultiSamples(0)  # Disable multi-sampling for depth peeling
renderWindow.AddRenderer(renderer)

style = vtk.vtkInteractorStyleTrackballCamera()

renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
renderWindowInteractor.SetInteractorStyle(style)

# Create an axes actor
axes = vtk.vtkAxesActor()
axes.SetTotalLength(10e-3, 10e-3, 10e-3)  # Length of the axes
axes.SetShaftTypeToCylinder()  # Set the style of the axes
axes.SetXAxisLabelText('')
axes.SetYAxisLabelText('')
axes.SetZAxisLabelText('')
renderer.AddActor(axes)     # Add the axes actor to the renderer

# Rotate the camera by 180 degrees around the x-axis
camera = renderer.GetActiveCamera()
camera.Roll(180)  # Rotate around the camera's view axis
camera.Azimuth(180)  # Rotate around the x-axis in the world coordinates

#add_sphere(renderer, center=(20e-3, 0, 0), radius=2e-3)
#add_cuboid(renderer, center=(-20e-3, 0, 0), size=[1e-3, 2e-3, 3e-3])
#add_frustum(renderer, position=(0, 15e-3, 0), back_dims=(50e-3, 30e-3), height=100e-3)
#add_line(renderer, start_point=(0, 0, 0), end_point=(10e-3, 10e-3, 10e-3), color=(1, 0, 0))
#add_circle(renderer, center=(0, 0, 0), radius=5e-3, direction=(1, 0, 0), color=(0, 1, 0), thickness=2.0, opacity=0.25)
#add_triangle(renderer, point1=(0, 0, 0), point2=(10e-3, 25e-3, 50e-3), point3=(5e-3, 10e-3, 100e-3), color=(0, 0, 1), opacity=0.5)
#add_cylinder(renderer, center=(-100e-3, -100e-3, 0), radius=5e-3, direction=(0, 0, -1), height=100e-3, color=(0, 0, 0), opacity=0.5, resolution=100)
#add_rectangle(renderer, point1=(0, 0, 0), point2=(100e-3, 0, 0), point3=(100e-3, 100e-3, 0), point4=(0, 100e-3, 0), color=(0, 0, 1), opacity=0.5)

[camera_frustum, ldisp_frustum, rdisp_frustum] = supernova.get_frustums()
add_frustum(renderer, camera_frustum, color=(0, 1, 1))
add_frustum(renderer, ldisp_frustum, color=(0, 1, 0))
add_frustum(renderer, rdisp_frustum, color=(1, 1, 0))

[pt2d_cam, depth, pt2d_ldisp, pt2d_rdisp] = supernova.compute_projection_error(np.array([supernova.K_cam[0, 2], supernova.K_cam[1, 2]]), [250e-3, 5000e-3, 10])
print(f'pt2d_cam: {pt2d_cam}')
print(f'depth: {depth}')
print(f'pt2d_ldisp: {pt2d_ldisp}')
print(f'pt2d_rdisp: {pt2d_rdisp}')


'''pt2_0 = np.array([supernova.K_cam[0, 2], supernova.K_cam[1, 2]])
depth = 5000e-3
pt3_w_cam_0 = np.linalg.inv(supernova.R_cam)@(np.array([0, 0, 0]).reshape(3, 1) - supernova.t_w_cam)
pt3_w_cam_1= supernova.backproject2world(pt2_0, supernova.K_cam, supernova.R_cam, supernova.t_w_cam, depth)
add_line(renderer, start_point=pt3_w_cam_0, end_point=pt3_w_cam_1, color=(0, 0, 0), thickness = 2.0)
add_sphere(renderer, center=pt3_w_cam_1, radius=5e-3, color=(0, 0, 0), opacity=1)
pt3_w_ldisp_0 = np.linalg.inv(supernova.R_ldisp)@(np.array([0, 0, 0]).reshape(3, 1) - supernova.t_w_ldisp)
add_line(renderer, start_point=pt3_w_ldisp_0, end_point=pt3_w_cam_1, color=(0, 0, 0), thickness = 2.0)
pt3_w_rdisp_0 = np.linalg.inv(supernova.R_rdisp)@(np.array([0, 0, 0]).reshape(3, 1) - supernova.t_w_rdisp)
add_line(renderer, start_point=pt3_w_rdisp_0, end_point=pt3_w_cam_1, color=(0, 0, 0), thickness = 2.0)'''

# Render and start interaction
renderWindow.Render()
renderWindowInteractor.Start()

## Calculate Camera-Display Projection Uncertainy

In [ ]:
import matplotlib.pyplot as plt
from projection import projection

supernova = projection(headset_params, cam_params, disp_params)

## Cast a ray from camera optical center to point on image plane
###pt2d_cam_1 = np.array([-1500, -2000])
#pt2d_cam_1 = np.array([0, 0])
#pt2d_cam_1 = np.array([1500, 2000])
###ray_cam_1 = compute_ray(K_cam, pt2d_cam_1)
###print(f'ray_cam_1: {ray_cam_1.shape}')

## Compute coordinates of points along ray in camera coordindate system
###d_min, d_max, n_bins = 0.25, 10.0, 50
###depths = np.array(np.linspace(d_min, d_max, n_bins)).reshape(1, n_bins)
# print(f'depths: {depths}')
# print(f'depths: {depths.shape}')
# print(f'ray_cam_1: {ray_cam_1.shape}')
# print(f'pt3d_cam_1: {pt3d_cam_1.shape}')
###pt3d_cam_1 = ray_cam_1 * depths
# print(f'Points along ray at depths: {pt3d_cam_1}')

## Compute coordinates of points along ray in world coordinate system
# print(f'R_cam: {R_cam.shape}')
# print(f'pt3d_cam_1: {pt3d_cam_1.shape}')
# print(f't_w_cam: {t_w_cam.shape}')
###pt3d_world_1 = np.linalg.inv(R_cam)@(pt3d_cam_1 - t_w_cam)
#print(f'pt3d_world_1: {pt3d_world_1.shape}')
# print(f'pt3d_cam_1: {pt3d_cam_1}')
# print('\r\n')
# print(f'pt3d_world_1: {pt3d_world_1}')

## Compute coordinates of points along ray in displays coordinate systems
# print(f'R_ldisp: {R_ldisp.shape}')
# print(f'pt3d_world_1: {pt3d_world_1.shape}')
# print(f't_w_ldisp: {t_w_ldisp.shape}')
###pt3d_ldisp_1 = R_ldisp@pt3d_world_1 + t_w_ldisp
###pt3d_rdisp_1 = R_rdisp@pt3d_world_1 + t_w_rdisp
# print(f'pt3d_ldisp_1: {pt3d_ldisp_1.shape}')
# print('\r\n')
# print(f'pt3d_world_1: {pt3d_world_1}')
# print('\r\n')
# print(f'pt3d_ldisp_1: {pt3d_ldisp_1}')

## Compute 2d coordinate in image plane from 3d coordinate system
# print(f'pt3d_ldisp_1{pt3d_ldisp_1.shape}')
###pt2d_ldisp_1 = project_point(pt3d_ldisp_1, K_disp)
###pt2d_rdisp_1 = project_point(pt3d_rdisp_1, K_disp)
#print(f'pt2d_ldisp_1{pt2d_ldisp_1.shape}')
#print(f'pt2d_ldisp_1{pt2d_ldisp_1}')
#print(f'pt2d_rdisp_1{pt2d_rdisp_1}')

'''x = depths[0].tolist()
y = []
for i, depth in enumerate(depths.T):
    euc_dist = np.sqrt(np.power((pt2d_ldisp_1[0, 0] - pt2d_ldisp_1[0, i]), 2) + np.power((pt2d_ldisp_1[1, 0] - pt2d_ldisp_1[1, i]), 2))
    y.append(euc_dist)
print(x)
print(y)
plt.figure(figsize=(10, 5))
plt.plot(x, y, marker='o', linestyle='-', color='b')
plt.title(f'Projection Error on Display from fixed Camera Point\nCamera point: {pt2d_cam_1}\n')
plt.xlabel('Depth [m]')
plt.ylabel('Pixel error')
plt.grid(True)
plt.show()'''